In [15]:
using HorizonSideRobots

In [20]:
r=Robot(animate=true)

Robot(HorizonSideRobots.SituationDatas.SituationData((0x000000000000000b, 0x000000000000000c), 1.0, true, (11, 1), [341 366 … -263 -256; 271 -69 … 33 -39; … ; 485 382 … 253 174; 380 -9 … 10 192], Set{Tuple{Int64, Int64}}(), Set{HorizonSide}[Set() Set() … Set() Set(); Set() Set() … Set() Set(); … ; Set() Set() … Set() Set(); Set() Set() … Set() Set()], (Scene (780px, 715px):
  0 Plots
  1 Child Scene:
    └ Scene (780px, 715px), Axis (6 plots))), true)

In [18]:
function moves!(r::Robot, side::HorizonSide, n_steps::Int)
    for i in 1:n_steps
        move!(r, side)
    end
end

moves! (generic function with 1 method)

In [22]:
function get_left_down_angle_modified!(r::Robot)::Vector{Tuple{HorizonSide, Int}}
    steps = []
    while !(isborder(r, West) && isborder(r, Sud))
        steps_to_West = move_until_border!(r, West)
        steps_to_Sud = move_until_border!(r, Sud)
        push!(steps, (West, steps_to_West))
        push!(steps, (Sud, steps_to_Sud))
    end
    return steps
end

get_left_down_angle_modified! (generic function with 1 method)

In [23]:
function move_if_possible!(r::Robot, side::HorizonSide)::Bool
    if !isborder(r, side)
        move!(r, side)
        return true
    end
    return false
end

move_if_possible! (generic function with 1 method)

In [24]:
function inversed_path(path::Vector{Tuple{HorizonSide, Int}})::Vector{Tuple{HorizonSide, Int}}
    inv_path = []
    for step in path
        inv_step = (inverse_side(step[1]), step[2])
        push!(inv_path, inv_step)
    end
    reverse!(inv_path)
    return inv_path
end

inversed_path (generic function with 1 method)

In [6]:
function make_way!(r::Robot, path::Vector{Tuple{HorizonSide, Int}})
    for step in path
        moves!(r, step[1], step[2])
    end
end

make_way! (generic function with 1 method)

In [25]:
function make_way_back!(r::Robot, path::Vector{Tuple{HorizonSide, Int}})
    inv_path = inversed_path(path)
    make_way!(r, inv_path)
end

make_way_back! (generic function with 1 method)

In [26]:
function mark_inner_rectangle!(r::Robot)
    steps = get_left_down_angle_modified!(r)

    while isborder(r, Sud) && !isborder(r, Ost)
        move_until_border!(r, Nord)
        move!(r, Ost)
        while !isborder(r, Ost) && move_if_possible!(r, Sud) end
    end

    for sides in [(Sud, Ost), (Ost, Nord), (Nord, West), (West, Sud)]
        side_to_move, side_to_border = sides
        while isborder(r, side_to_border)
            putmarker!(r)
            move!(r, side_to_move)
        end
        putmarker!(r)
        move!(r, side_to_border)
    end

    get_left_down_angle_modified!(r)
    make_way_back!(r, steps)
end

mark_inner_rectangle! (generic function with 1 method)

In [27]:
function move_until_border!(r::Robot, side::HorizonSide)::Int
    n_steps = 0
    while !isborder(r, side)
        n_steps += 1
        move!(r, side)
    end
    return n_steps
end

move_until_border! (generic function with 1 method)

In [28]:
function inverse_side(side::HorizonSide)::HorizonSide
    inv_side = HorizonSide((Int(side) + 2) % 4)
    return inv_side
end

inverse_side (generic function with 1 method)

In [29]:
mark_inner_rectangle!(r)